In [1]:
import pandas as pd
import Bio
from Bio import SeqIO
from Bio import Entrez

In [44]:
#Species list
hanta_species = pd.read_csv('../data/ncbi_virus/associatedViralSpecies_cophylo.txt', sep='\t', header=None)
#rename the column
hanta_species.columns = ['species']

#Arha data
arha = pd.read_csv('../data/ncbi_virus/hantaviridae_ncbi_03062025.csv')
arha.columns = arha.columns.str.lower()
#Keep the L segment only
L_data = arha[arha['genbank_title'].str.contains(r"polymerase|segment L|L segment|Large", case=False, na=False)]
M_data = arha[arha['genbank_title'].str.contains(r"glycoprotein|segment M|M segment|Medium|medium|G1|G2", case=False, na=False)]
S_data = arha[arha['genbank_title'].str.contains(r"Nucleoprotein|nucleprotein|nucleocapsid|Nucleocapsid|segment S|S segment|Small|small", case=False, na=False)]
#Filter L and M from S data
S_data = S_data[S_data['segment'] != 'L']
S_data = S_data[S_data['segment'] != 'M']

#Group the arha data based on the species and find the most complete sequence

def select_longest_sequence(df, species_col="species", length_col="length", min_length=200, max_length=8000):
    # Convert the length column to numeric and drop rows with missing species or length values
    df[length_col] = pd.to_numeric(df[length_col], errors='coerce')
    df = df.dropna(subset=[species_col, length_col])
    
    def get_best_sequence(group):
        # Filter for sequences within the desired length range
        in_range = group[(group[length_col] >= min_length) & (group[length_col] <= max_length)]
        if not in_range.empty:
            # If sequences within the range exist, return the one with the maximum length
            return in_range.loc[in_range[length_col].idxmax()]
        else:
            # Otherwise, return the sequence with the maximum length overall (if available)
            if group[length_col].notna().any():
                return group.loc[group[length_col].idxmax()]
            else:
                return None

    result = df.groupby(species_col, group_keys=False).apply(get_best_sequence)
    result = result.dropna(subset=[length_col])
    return result

compare_datasets = [L_data, M_data, S_data]
for dataset in compare_datasets:
    selected_sequences = select_longest_sequence(dataset)
    print(len(selected_sequences[selected_sequences['species'].isin(hanta_species['species'])]))

#Since S segment has the most diversity, we'll use it

selected_sequences = select_longest_sequence(S_data)
selected_sequences = selected_sequences[selected_sequences['species'].isin(hanta_species['species'])]
accession_numbers = selected_sequences['accession'].tolist()

#Download the sequences
Entrez.email = "ricardo.rivero@wsu.edu"
handle = Entrez.efetch(db="nucleotide", id=accession_numbers, rettype="fasta", retmode="text")
records = SeqIO.parse(handle, "fasta")

#write the sequences to a file
with open('../data/ncbi_virus/hanta_S.fasta', 'w') as f:
    SeqIO.write(records, f, "fasta")

selected_sequences

/var/folders/8v/6y3r5h_x0dv5t5875drggrtc0000gn/T/ipykernel_43714/1043120449.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[length_col] = pd.to_numeric(df[length_col], errors='coerce')
/var/folders/8v/6y3r5h_x0dv5t5875drggrtc0000gn/T/ipykernel_43714/1043120449.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[length_col] = pd.to_numeric(df[length_col], errors='coerce')


57
61
70


,accession,organism_name,genbank_refseq,assembly,sra_accession,submitters,organization,org_location,release_date,isolate,...,publications,geo_location,country,usa,host,tissue_specimen_source,collection_date,biosample,bioproject,genbank_title
2,NC_077668,Orthohantavirus puumalaense,RefSeq,GCF_002829765.1,NaN,"Stohwasser,R., Giebel,L.B., Zoller,L., Bautz,E...","National Center for Biotechnology Information,...",USA,2023-05-06,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRJNA485481,Puumala virus CG1820 segment S nucleocapsid pr...
3,NC_077669,Orthohantavirus sinnombreense,RefSeq,GCF_002830985.1,NaN,"Chizhikov,V.E., Spiropoulou,C.F., Morzunov,S.P...","National Center for Biotechnology Information,...",USA,2023-05-06,NM R11,...,1.0,NaN,NaN,NaN,Peromyscus maniculatus,NaN,NaN,NaN,PRJNA485481,Sin Nombre virus (NM R11) RNA S segment encodi...
8,NC_078263,Xuan son virus,RefSeq,GCF_018594905.1,NaN,NaN,"National Center for Biotechnology Information,...",USA,2023-05-06,PR15,...,NaN,"China: Yunnan province, Puer city",China,NaN,Hipposideros pomona,lung,2013,NaN,PRJNA485481,"Xuan son virus isolate PR15 segment S, complet..."
9,NC_078483,Lena virus,RefSeq,GCF_018596435.1,NaN,"Yashina,L.N., Kartashov,M.Y., Wang,W., Li,K., ...","National Center for Biotechnology Information,...",USA,2023-05-06,NaN,...,1.0,Russia: Khabarovsk Krai,Russia,NaN,Sorex caecutiens,lung,2008-02,NaN,PRJNA485481,Lena River virus strain Khekhtsir-Sc67/Russia/...
18,NC_055375,Brno virus,RefSeq,GCF_013086645.1,NaN,"Strakova,P., Dufkova,L., Sirmarova,J., Salat,J...","National Center for Biotechnology Information,...",USA,2021-06-01,NaN,...,1.0,Czech Republic,Czech Republic,NaN,Nyctalus noctula,liver,2012,NaN,PRJNA485481,Brno virus strain 7/2012/CZE nucleocapsid prot...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15335,M34881,Sapporo rat virus,GenBank,set:M34881,NaN,"Arikawa,J., Lapenotiere,H.F., Iacono-Connors,L...",NaN,NaN,1993-08-03,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sapporo rat virus mRNA S segment, encoding an ..."
15339,M14626,Orthohantavirus hantanense,GenBank,set:M14626,NaN,"Schmaljohn,C.S., Jennings,G.B., Hay,J., Dalrym...",NaN,NaN,1993-08-02,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hantaan virus S segment encoding nucleocapsid ...
15345,L08804,Orthohantavirus puumalaense,GenBank,set:503a14cd9dd548e3916edc83af80d5a439809964,NaN,"Xiao,S.Y., Spik,K.W., Li,D., Schmaljohn,C.S.",NaN,NaN,1993-07-26,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hantavirus Puumala nucleocapsid protein, compl..."
15347,L11347,Orthohantavirus puumalaense,GenBank,set:L11347,NaN,"Xiao,S.Y., Spik,K.W., Li,D., Schmaljohn,C.S.",NaN,NaN,1993-06-12,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hantavirus Puumala P360 nucleocapsid protein m...


In [38]:
#Align using mafft
import subprocess
subprocess.run(["mafft", "--genafpair", "--maxiterate 16", "-reorder","../data/ncbi_virus/hanta_S.fasta", ">", "../data/ncbi_virus/hanta_S_aligned.fasta"], shell=True)

awk: cmd. line:11: fatal: cannot redirect to `/dev/tty': Device not configured


CompletedProcess(args=['mafft', '--auto', '../data/ncbi_virus/hanta_S.fasta', '>', '../data/ncbi_virus/hanta_S_aligned.fasta'], returncode=0)